In [1]:
import requests
import lxml.html as lh
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine #untuk membuat sql & commandnya berjalan dengan data kita

url='http://pokemondb.net/pokedex/all'#'https://datatables.net/'
page = requests.get(url) #menyimpan hasil permintaan dari website
doc = lh.fromstring(page.content)
#Parse data

tr_elements = doc.xpath('//tr')
#membuat list kosong untuk penyimpanan data tabel
col=[]
i=0
for t in tr_elements[0]: #setiap baris disimpan pada list kosong
    i+=1
    name=t.text_content()
    #print('%d:"%s"'%(i,name)) #coba cek ukuran tampilan dari kolom jika sama = sesuai (data berada di tabel yang kita inginkan)
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    T=tr_elements[j] #T menyimpan baris yang ke-j
    if len(T)!=10: #check data tabel memiliki 10 kolom,jika lebih stop, agar data yang di ambil sesuai dan tidak berasal dr tabel
        break
    
    #i sbg nomor kolom
    i=0
    
    #mengiterasikan sesuai nomor kolom,berurutan
    for t in T.iterchildren():
        data=t.text_content() 
        #check jika kolom kosong, maka diisi
        if i>0:
        #ubah data text menjadi int agar bisa di lakukan perhitungan matematis
            try:
                data=int(data)
            except:
                pass
        #masukkan data di ujung "append" / data dimasukkan diujung karena list masih kosong
        col[i][1].append(data)
        i+=1 #setelah disini masuk ke baris selanjutnya

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
engine = create_engine('sqlite:///:memory:')
df.to_sql('data_table',engine)
#df #memiliki 925 data,926 termasuk baris informasi data



In [2]:
#checkupdate data
res=pd.read_sql_query('SELECT * FROM data_table',engine)
res

,index,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,3,003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,4,004,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
1024,1024,888,Zacian Hero of Many Battles,Fairy,670,92,130,115,80,115,138
1025,1025,889,Zamazenta Crowned Shield,Fighting Steel,720,92,130,145,80,145,128
1026,1026,889,Zamazenta Hero of Many Battles,Fighting,670,92,130,115,80,115,138
1027,1027,890,Eternatus,Poison Dragon,690,140,85,95,145,95,130


In [3]:
ratahp = pd.read_sql_query('SELECT avg(HP) FROM data_table',engine)
ratahp #cari rata rata hp

,avg(HP)
0,69.597668


In [4]:
#menambahkan baris pada tabel
addbaris = """INSERT INTO data_table VALUES (0,"0", "Pokemonjadijadian","Unknown","999","999","999","999","999","999","999");"""
sql.execute(addbaris,engine)

In [5]:
res=pd.read_sql_query('SELECT*FROM data_table order by Speed desc', engine)
res

,index,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0,0,Pokemonjadijadian,Unknown,999,999,999,999,999,999,999
1,464,386,Deoxys Speed Forme,Psychic,600,50,95,90,95,90,180
2,345,291,Ninjask,Bug Flying,456,61,90,45,50,50,160
3,923,795,Pheromosa,Bug Fighting,570,71,137,37,137,37,151
4,84,065,Alakazam Mega Alakazam,Psychic,600,55,50,65,175,105,150
...,...,...,...,...,...,...,...,...,...,...,...
1025,696,597,Ferroseed,Grass Steel,305,44,50,91,24,86,10
1026,960,829,Gossifleur,Grass,250,40,40,60,40,60,10
1027,257,213,Shuckle,Bug Rock,505,20,10,230,10,230,5
1028,528,446,Munchlax,Normal,390,135,85,40,40,85,5


In [6]:
delbaris = """DELETE FROM data_table WHERE Name='Pokemonjadijadian'"""
sql.execute(delbaris,engine)
#DELETE BARIS DARI NAMA


In [7]:
res=pd.read_sql_query('SELECT*FROM data_table order by Speed desc', engine)
res

,index,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,464,386,Deoxys Speed Forme,Psychic,600,50,95,90,95,90,180
1,345,291,Ninjask,Bug Flying,456,61,90,45,50,50,160
2,923,795,Pheromosa,Bug Fighting,570,71,137,37,137,37,151
3,84,065,Alakazam Mega Alakazam,Psychic,600,55,50,65,175,105,150
4,131,101,Electrode,Electric,490,60,50,70,80,80,150
...,...,...,...,...,...,...,...,...,...,...,...
1024,696,597,Ferroseed,Grass Steel,305,44,50,91,24,86,10
1025,960,829,Gossifleur,Grass,250,40,40,60,40,60,10
1026,257,213,Shuckle,Bug Rock,505,20,10,230,10,230,5
1027,528,446,Munchlax,Normal,390,135,85,40,40,85,5


In [8]:
#data yang unik
available_Type= df['Type'].unique()
available_Type

array(['Grass Poison', 'Fire ', 'Fire Flying', 'Fire Dragon', 'Water ',
       'Bug ', 'Bug Flying', 'Bug Poison', 'Normal Flying', 'Normal ',
       'Dark Normal', 'Poison ', 'Electric ', 'Electric Psychic',
       'Ground ', 'Ice Steel', 'Poison Ground', 'Fairy ', 'Ice ',
       'Ice Fairy', 'Normal Fairy', 'Poison Flying', 'Bug Grass',
       'Ground Steel', 'Dark ', 'Steel ', 'Fighting ', 'Water Fighting',
       'Psychic ', 'Water Poison', 'Rock Ground', 'Rock Electric',
       'Psychic Fairy', 'Water Psychic', 'Electric Steel', 'Water Ice',
       'Poison Dark', 'Ghost Poison', 'Grass Psychic', 'Grass Dragon',
       'Fire Ghost', 'Poison Fairy', 'Ground Rock', 'Grass ',
       'Psychic Ice', 'Ice Psychic', 'Water Flying', 'Water Dark',
       'Rock Water', 'Rock Flying', 'Ice Flying', 'Electric Flying',
       'Dragon ', 'Dragon Flying', 'Psychic Fighting', 'Water Electric',
       'Fairy Flying', 'Psychic Flying', 'Electric Dragon', 'Water Fairy',
       'Rock ', 'Grass Flying'

In [9]:
#penjumlahan berdasarkan Type
count_Type = df.groupby(['Type']).size().reset_index(name='counts')
#print(count_Type)

In [10]:
#pengelompokan berdasarkan type dengan speed>100
res2=pd.read_sql_query('SELECT*FROM data_table WHERE Speed>100 ORDER BY Type ASC', engine)
#res2

In [11]:
#pengelompokan berdasarkan Type dan penjumlahan Speed
res = df.groupby(['Type'])['Speed'].sum().reset_index()
#res

In [12]:
df.head()
df.tail()
df.shape
df.info
df.describe

<bound method NDFrame.describe of         #                            Name            Type  Total   HP  Attack  \
0     001                       Bulbasaur    Grass Poison    318   45      49   
1     002                         Ivysaur    Grass Poison    405   60      62   
2     003                        Venusaur    Grass Poison    525   80      82   
3     003          Venusaur Mega Venusaur    Grass Poison    625   80     100   
4     004                      Charmander           Fire     309   39      52   
...   ...                             ...             ...    ...  ...     ...   
1024  888     Zacian Hero of Many Battles          Fairy     670   92     130   
1025  889        Zamazenta Crowned Shield  Fighting Steel    720   92     130   
1026  889  Zamazenta Hero of Many Battles       Fighting     670   92     130   
1027  890                       Eternatus   Poison Dragon    690  140      85   
1028  890             Eternatus Eternamax   Poison Dragon   1125  255     1

In [13]:
#penjumlahan berdasarkan Speed
count_Type = df.groupby(['Speed']).size().reset_index(name='counts')
#count_Type

In [14]:
#penjumlahan berdasarkan Total
count_Total = df.groupby(['Total']).size().reset_index(name='counts')
#count_Total

In [15]:
#pengelompokan berdasarkan Type dan penjumlahan Total
res = df.groupby(['Type'])['Total'].sum().reset_index()
#res